# Test Model

In [1]:
import os
from pathlib import Path
import sys
sys.path.append(str(Path(os.getcwd()).parent))

from settings.global_settings import GlobalSettings

config = GlobalSettings.get_config(
    config_file = "../config.ini",
    secrets_file = "../secrets.ini"
)
from dataset.video_loader import VideoDataLoader
from dataset.video_dataset import VideoDataset
from model.training_loop import train, EarlyStoppingParams
from model.multimodal_har_model import MultiModalHARModel

2025-10-25 20:19:23,408 - INFO - Sentry DSN set to: https://f4f21cc936b3ba9f5dbc1464b7a40ea4@o4504168838070272.ingest.us.sentry.io/4506464560414720
2025-10-25 20:19:23,409 - INFO - Sentry initialized with environment: development


Loading config...
Loading secrets...


## Initializing Training

**Creating Dataloaders**

In [2]:
TRAIN_DIR = os.path.join(
    config.model_settings.video_data_dir,
    "train"
)
TEST_DIR = os.path.join(
    config.model_settings.video_data_dir,
    "validation"
)
VALIDATION_DIR = os.path.join(
    config.model_settings.video_data_dir,
    "test"
)

train_video_data_loader = VideoDataLoader(
    path=TRAIN_DIR,
)
test_video_data_loader = VideoDataLoader(
    path=TEST_DIR,
)
validation_video_data_loader = VideoDataLoader(
    path=VALIDATION_DIR,
)

train_dataset = VideoDataset(
    video_data_loader=train_video_data_loader,
    normalization_type="across_frames",
)
test_dataset = VideoDataset(
    video_data_loader=test_video_data_loader,
    normalization_type="across_frames",
)
validation_dataset = VideoDataset(
    video_data_loader=validation_video_data_loader,
    normalization_type="across_frames",
)

len(train_dataset)
for _ in train_dataset:
    pass
len(train_dataset.labels_map)

len(test_dataset)
for _ in test_dataset:
    pass

len(validation_dataset)
for _ in validation_dataset:
    pass


display(len(test_dataset.labels_map))
display(len(validation_dataset.labels_map))
display(len(train_dataset.labels_map))

2025-10-25 20:19:25,459 - INFO - [VideoDataLoader] Loding action videos for action: a01
2025-10-25 20:19:26,073 - INFO - [VideoDataLoader] Loding action videos for action: a02
2025-10-25 20:19:26,813 - INFO - [VideoDataLoader] Loding action videos for action: a03
2025-10-25 20:19:27,466 - INFO - [VideoDataLoader] Loding action videos for action: a04
2025-10-25 20:19:28,576 - INFO - [VideoDataLoader] Loding action videos for action: a05
2025-10-25 20:19:29,194 - INFO - [VideoDataLoader] Loding action videos for action: a06
2025-10-25 20:19:29,774 - INFO - [VideoDataLoader] Loding action videos for action: a08
2025-10-25 20:19:31,225 - INFO - [VideoDataLoader] Loding action videos for action: a09
2025-10-25 20:19:32,442 - INFO - [VideoDataLoader] Loding action videos for action: a11
2025-10-25 20:19:33,247 - INFO - [VideoDataLoader] Loding action videos for action: a12
2025-10-25 20:19:38,367 - INFO - [VideoDataLoader] Loding action videos for action: a01
2025-10-25 20:19:38,395 - INFO -

10

10

10

**Splitting Train and Test Datasets**

In [3]:
# num_total = len(train_dataset)
# num_train = int(0.8 * num_total)
# num_test = num_total - num_train
# train_dataset, test_dataset = random_split(train_dataset, [num_train, num_test])

**Creating Model**

In [4]:

har_model = MultiModalHARModel(
    obj_in=train_dataset[0].graphs_objects[0].x.shape[1],
    joint_in=train_dataset[0].graphs_joints[0].x.shape[1],
    gat_hidden=128,
    gat_out=128,
    temporal_hidden=128,
    num_classes=len(train_dataset.labels_map), 
    dropout=0.5,
)

**Create Evaluate Function For Early Stopping**

In [5]:
def evaluate(model, dataset):
    import torch
    device = 'mps'
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for i in range(len(dataset)):
            sample = dataset[i]
            label = sample.label.to(device)

            # Move all graph tensors to device
            graphs_objects = [g.to(device) for g in sample.graphs_objects]
            graphs_joints = [g.to(device) for g in sample.graphs_joints]

            # Forward pass
            output = model(graphs_objects, graphs_joints)

            # Compute prediction
            if output.dim() == 1:
                predicted = torch.argmax(output).unsqueeze(0)
            else:
                _, predicted = torch.max(output, dim=1)

            correct += (predicted == label).sum().item()
            total += 1

    accuracy = 100 * correct / total
    return accuracy


## Training Model

**Early Stopping**

In [6]:
early_stopping_params = EarlyStoppingParams(
    patience=5,
    min_delta=1e-4,
    mode='max',
    evaluation_function=evaluate,
    evaluation_dataset=validation_dataset,
)

**Training**

In [7]:
train_history = train(
    model=har_model,
    video_dataset=train_dataset,
    device='mps',
    epochs=25,
    early_stopping=early_stopping_params,
)

2025-10-25 20:19:40,682 - INFO - Starting training loop...
2025-10-25 20:19:40,722 - INFO - Using early stopping
Epoch 1/25: 100%|██████████| 1176/1176 [06:40<00:00,  2.94it/s]
2025-10-25 20:26:21,053 - INFO - Evaluating for early stopping...
2025-10-25 20:26:45,602 - INFO - Epoch 1/25, Loss: 1.9703
Epoch 2/25: 100%|██████████| 1176/1176 [06:25<00:00,  3.05it/s]
2025-10-25 20:33:11,205 - INFO - Evaluating for early stopping...
2025-10-25 20:33:32,582 - INFO - Epoch 2/25, Loss: 1.5095
Epoch 3/25: 100%|██████████| 1176/1176 [06:26<00:00,  3.04it/s]
2025-10-25 20:39:59,164 - INFO - Evaluating for early stopping...
2025-10-25 20:40:21,179 - INFO - Epoch 3/25, Loss: 1.3600
Epoch 4/25: 100%|██████████| 1176/1176 [06:23<00:00,  3.07it/s]
2025-10-25 20:46:44,329 - INFO - Evaluating for early stopping...
2025-10-25 20:47:04,970 - INFO - Epoch 4/25, Loss: 1.1955
Epoch 5/25: 100%|██████████| 1176/1176 [06:05<00:00,  3.22it/s]
2025-10-25 20:53:10,650 - INFO - Evaluating for early stopping...
2025-

**Saving Model**

In [8]:
har_model.save(
    training_history=train_history
)

2025-10-25 22:30:46,802 - INFO - Saving model to /Volumes/KODAK/masters/model/validation_datasets/NW-UCLA/model/har_model_v0.0.0_nw_ucla_2025-10-25 22:30:46.802204.pht...
2025-10-25 22:30:46,908 - INFO - Model saved successfully.


In [9]:
import torch
from datetime import datetime

model_settings = config.model_settings
save_dir = "~/masters-models"
os.makedirs(save_dir, exist_ok=True)

save_path = os.path.join(
    save_dir,
    f"har_model_{model_settings.model_version}_{model_settings.dataset_prefix}_100perc_{datetime.now()}.pht"
)
torch.save({
"model_state_dict": har_model.state_dict(),
"training_history": train_history,
"model_config": har_model._model_config,
}, save_path)


## Running tests

**Accuracy**

In [10]:
accuracy_evaluation = evaluate(har_model, test_dataset)
print(f"Test Accuracy: {accuracy_evaluation:.2f}%")

Test Accuracy: 73.03%
